In [1]:
import pandas as pd, numpy as np
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import pickle
import sys
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score

# Load Train Data and Labels

In [2]:
IS_VALIDATION = False
IS_SUBMIT = True

In [3]:
dtypes = {"session_id": 'int64',
          "index": np.int16,
          "elapsed_time": np.int32,
          "event_name": 'category',
          "name": 'category',
          "level": np.int8,
          "page": np.float16,
          "room_coor_x": np.float16,
          "room_coor_y": np.float16,
          "screen_coor_x": np.float16,
          "screen_coor_y": np.float16,
          "hover_duration": np.float32,
          "text": 'category',
          "fqid": 'category',
          "room_fqid": 'category',
          "text_fqid": 'category',
          "fullscreen": np.int8,
          "hq": np.int8,
          "music": np.int8,
          "level_group": 'category'
          }
use_col = ['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level', 'page',
           'room_coor_x', 'room_coor_y', 'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid', 'level_group']

In [4]:
targets = pd.read_csv('/kaggle/input/predict-student-performance-from-game-play/train_labels.csv')
targets['session'] = targets.session_id.apply(lambda x: int(x.split('_')[0]) )
targets['q'] = targets.session_id.apply(lambda x: int(x.split('_')[-1][1:]) )
print( targets.shape )
targets.head()

(424116, 4)


,session_id,correct,session,q
0,20090312431273200_q1,1,20090312431273200,1
1,20090312433251036_q1,0,20090312433251036,1
2,20090312455206810_q1,1,20090312455206810,1
3,20090313091715820_q1,0,20090313091715820,1
4,20090313571836404_q1,1,20090313571836404,1


In [5]:
feature_df = pd.read_csv('/kaggle/input/featur/feature_sort.csv')

# Feature Engineer

In [6]:
def delt_time_def(df):
    df.sort_values(by=['session_id', 'elapsed_time'], inplace=True)
    df['d_time'] = df['elapsed_time'].diff(1)
    df['d_time'].fillna(0, inplace=True)
    df['delt_time'] = df['d_time'].clip(0, 103000)
    df['delt_time_next'] = df['delt_time'].shift(-1)
    return df

In [7]:
def feature_engineer(train, kol_f):
    global kol_col, kol_col_max
    kol_col = 9
    kol_col_max = 11+kol_f*2
    col = [i for i in range(0,kol_col_max)]
    new_train = pd.DataFrame(index=train['session_id'].unique(), columns=col, dtype=np.float16)  
    new_train[10] = new_train.index # "session_id"    

    new_train[0] = train.groupby(['session_id'])['d_time'].quantile(q=0.3)
    new_train[1] = train.groupby(['session_id'])['d_time'].quantile(q=0.8)
    new_train[2] = train.groupby(['session_id'])['d_time'].quantile(q=0.5)
    new_train[3] = train.groupby(['session_id'])['d_time'].quantile(q=0.65)
    new_train[4] = train.groupby(['session_id'])['hover_duration'].agg('mean')
    new_train[5] = train.groupby(['session_id'])['hover_duration'].agg('std')    
    new_train[6] = new_train[10].apply(lambda x: int(str(x)[:2])).astype(np.uint8) # "year"
    new_train[7] = new_train[10].apply(lambda x: int(str(x)[2:4])+1).astype(np.uint8) # "month"
    new_train[8] = new_train[10].apply(lambda x: int(str(x)[4:6])).astype(np.uint8) # "day"
    new_train[9] = new_train[10].apply(lambda x: int(str(x)[6:8])).astype(np.uint8) + new_train[10].apply(lambda x: int(str(x)[8:10])).astype(np.uint8)/60
    new_train[10] = 0
    new_train = new_train.fillna(-1)
    
    return new_train

In [8]:
def feature_next_t(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:       
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska].groupby(['session_id'])['index'].count()
    return new_train

In [9]:
def feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    col1 = row_f['col1']
    val1 = row_f['val1']
    maska = (train[col1] == val1)
    if row_f['kol_col'] == 1:      
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = maska & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

In [10]:
def experiment_feature_next_t_otvet(row_f, new_train, train, gran_1, gran_2, i):
    global kol_col
    kol_col +=1
    if row_f['kol_col'] == 1: 
        maska = train[row_f['col1']] == row_f['val1']
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()          
    elif row_f['kol_col'] == 2: 
        col2 = row_f['col2']
        val2 = row_f['val2']
        maska = (train[col1] == val1) & (train[col2] == val2)        
        new_train[kol_col] = train[maska]['delt_time_next'].sum()
        if gran_1:
            kol_col +=1
            new_train[kol_col] = train[maska]['delt_time'].mean()
        if gran_2:
            kol_col +=1
            new_train[kol_col] = train[maska]['index'].count()
    return new_train

In [11]:
def feature_quest_otvet(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    g1 = 0.7 
    g2 = 0.3 

    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f):         
        row_f = feature_q.loc[i]
        new_train = feature_next_t_otvet(row_f, new_train, train, i < gran1, i <  gran2, i) 
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

In [12]:
def feature_engineer_new(new_train, train, feature_q, kol_f):
    g1 = 0.7 
    g2 = 0.3 
    gran1 = round(kol_f * g1)
    gran2 = round(kol_f * g2)    
    for i in range(0, kol_f): 
        row_f = feature_q.loc[i]       
        new_train = feature_next_t(row_f, new_train, train, i < gran1, i <  gran2, i)         
    return new_train

In [13]:
def feature_quest(new_train, train, quest, kol_f):
    global kol_col
    kol_col = 9
    feature_q = feature_df[feature_df['quest'] == quest].copy()
    feature_q.reset_index(drop=True, inplace=True)
    new_train = feature_engineer_new(new_train, train, feature_q, kol_f)
    col = [i for i in range(0,kol_col+1)]
    return new_train[col]

In [14]:
def create_model(old_train, quests, models, list_kol_f):
    
    threshold = 0.63
    
    kol_quest = len(quests)
    # ITERATE THRU QUESTIONS
    for k, q in enumerate(quests):
        print('### quest ', q, end='')
        new_train = feature_engineer(old_train, list_kol_f[q])
        train_x = feature_quest(new_train, old_train, q, list_kol_f[q])
        print (' ---- ', 'train_q.shape = ', train_x.shape)
            
        # TRAIN DATA
        train_users = train_x.index.values
        train_y = targets.loc[targets.q==q].set_index('session').loc[train_users]

        # TRAIN MODEL 
        gkf = GroupKFold(n_splits=5)
        X = train_x.astype('float32').reset_index(drop=True).copy()
        y = train_y.reset_index(drop=True).copy()
        model = CatBoostClassifier(
                    n_estimators = 300,
                    learning_rate= 0.045,
                    depth = 6,
                    task_type='CPU')
        
        if IS_VALIDATION:
            scores = []
            for i, (train_index, test_index) in enumerate(gkf.split(X=X,y=y['correct'], groups=train_y.index)):
                X_Train, X_Test = X.loc[train_index,:], X.loc[test_index,:]
                y_Train, y_Test = y.loc[train_index,'correct'], y.loc[test_index,'correct']

                model.fit(X_Train, y_Train, verbose=False)
                y_pred = model.predict_proba(X_Test)[:,1]
                y_pred = (y_pred.reshape((-1))>threshold).astype('int')
                scores.append(f1_score(y_Test,y_pred, average='macro'))
                
            print("\nMean:",np.mean(scores))
        # SAVE MODEL, PREDICT VALID OOF
        if IS_SUBMIT:
            model.fit(X, y['correct'], verbose=False)
            models[q] = model
    print('***')
    
    return models

In [15]:
models = {}
best_threshold = 0.63

In [16]:
list_kol_f = {
    1:140,2:120,3:110,
    4:120, 5:220, 6:130, 7:110, 8:110, 9:100, 10:140, 11:120, 12:120, 13:120,
    14: 160, 15:160, 16:130, 17:140, 18:150             
             }

In [17]:
df0_4 = pd.read_csv('/kaggle/input/featur/train_0_4t.csv', dtype=dtypes) 
kol_lvl = (df0_4 .groupby(['session_id'])['level'].agg('nunique') < 5)
list_session = kol_lvl[kol_lvl].index
df0_4  = df0_4 [~df0_4 ['session_id'].isin(list_session)]
df0_4 = delt_time_def(df0_4)

quests_0_4 = [1,2,3] 

models = create_model(df0_4, quests_0_4, models, list_kol_f)
del df0_4

### quest  1 ----  train_q.shape =  (23562, 290)
### quest  2 ----  train_q.shape =  (23562, 250)
### quest  3 ----  train_q.shape =  (23562, 230)
***


In [18]:
df5_12 = pd.read_csv('/kaggle/input/featur/train_5_12t.csv', dtype=dtypes)
kol_lvl = (df5_12.groupby(['session_id'])['level'].agg('nunique') < 8)
list_session = kol_lvl[kol_lvl].index
df5_12 = df5_12[~df5_12['session_id'].isin(list_session)]
df5_12 = delt_time_def(df5_12)
quests_5_12 = [4, 5, 6, 7, 8, 9, 10, 11, 12, 13] 

# list_kol_f = {4:110, 5:220, 6:120, 7:110, 8:110, 9:100, 10:140, 11:120}

models = create_model(df5_12, quests_5_12, models, list_kol_f)
del df5_12

# Mean: 0.6706794887292276
# Mean: 0.6313519444517699
# Mean: 0.6414561905583858
# Mean: 0.6257191786189739
# Mean: 0.5619083089359364
# Mean: 0.6284340871564126
# Mean: 0.5691873364807394
# Mean: 0.6086697655564295

### quest  4 ----  train_q.shape =  (23561, 250)
### quest  5 ----  train_q.shape =  (23561, 450)
### quest  6 ----  train_q.shape =  (23561, 270)
### quest  7 ----  train_q.shape =  (23561, 230)
### quest  8 ----  train_q.shape =  (23561, 230)
### quest  9 ----  train_q.shape =  (23561, 210)
### quest  10 ----  train_q.shape =  (23561, 290)
### quest  11 ----  train_q.shape =  (23561, 250)
### quest  12 ----  train_q.shape =  (23561, 250)
### quest  13 ----  train_q.shape =  (23561, 250)
***


In [19]:
df13_22 = pd.read_csv('/kaggle/input/featur/train_13_22t.csv', dtype=dtypes) 
kol_lvl = (df13_22 .groupby(['session_id'])['level'].agg('nunique') < 10)
list_session = kol_lvl[kol_lvl].index
df13_22  = df13_22 [~df13_22 ['session_id'].isin(list_session)]
df13_22 = delt_time_def(df13_22)

quests_13_22 = [14, 15, 16, 17, 18] 
# list_kol_f = {14: 160, 15:160, 16:105, 17:140}

models = create_model(df13_22, quests_13_22, models, list_kol_f)

# Mean: 0.6343802011898173
# Mean: 0.5975221078208751
# Mean: 0.5011219447159772
# Mean: 0.5498305040970815

### quest  14 ----  train_q.shape =  (22986, 330)
### quest  15 ----  train_q.shape =  (22986, 330)
### quest  16 ----  train_q.shape =  (22986, 270)
### quest  17 ----  train_q.shape =  (22986, 290)
### quest  18 ----  train_q.shape =  (22986, 310)
***


**Infer Test Data**

In [20]:
if IS_SUBMIT:
    import jo_wilder

    try:
        jo_wilder.make_env.__called__ = False
        env.__called__ = False
        type(env)._state = type(type(env)._state).__dict__['INIT']
    except:
        pass

    env = jo_wilder.make_env()
    iter_test = env.iter_test()    

In [21]:
if IS_SUBMIT:
    import time
    g_end4 = 0
    g_end5 = 0

    list_q = {'0-4':quests_0_4, '5-12':quests_5_12, '13-22':quests_13_22}
    for (test, sam_sub) in iter_test:
        sam_sub['question'] = [int(label.split('_')[1][1:]) for label in sam_sub['session_id']]    
        grp = test.level_group.values[0]   
        sam_sub['correct'] = 1
        sam_sub.loc[sam_sub.question.isin([5, 8, 10, 13, 15]), 'correct'] = 0  
        old_train = delt_time_def(test[test.level_group == grp])

        for q in list_q[grp]:

            start4 = time.time()
            new_train = feature_engineer(old_train, list_kol_f[q])
            new_train = feature_quest_otvet(new_train, old_train, q, list_kol_f[q])
    #         new_train = feature_quest(new_train, old_train, q, kol_f)

            end4 = time.time() - start4
            g_end4 += end4

            start5 = time.time()        

            clf = models[q]
            p = clf.predict_proba(new_train.astype('float32'))[:,1]        

            end5 = time.time() - start5
            g_end5 += end5


            mask = sam_sub.question == q 
            x = int(p[0]>best_threshold)
            sam_sub.loc[mask,'correct'] = x      


        sam_sub = sam_sub[['session_id', 'correct']]      
        env.predict(sam_sub)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
